In [1]:
from math import *
import random

landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0


class robot:
    def __init__(self):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0;
        self.turn_noise    = 0.0;
        self.sense_noise   = 0.0;
    
    def set(self, new_x, new_y, new_orientation):
        if new_x < 0 or new_x >= world_size:
            raise ValueError('X coordinate out of bound')
        if new_y < 0 or new_y >= world_size:
            raise ValueError('Y coordinate out of bound')
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise ValueError ('Orientation must be in [0..2pi]')
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
    
    
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise);
        self.turn_noise    = float(new_t_noise);
        self.sense_noise   = float(new_s_noise);
    
    
    def sense(self):
        Z = []
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    
    def move(self, turn, forward):
        if forward < 0:
            raise ValueError ('Robot cant move backwards')
        
        # turn, and add randomness to the turning command
        orientation = self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        orientation %= 2 * pi
        
        # move, and add randomness to the motion command
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        x %= world_size    # cyclic truncate
        y %= world_size
        
        # set particle
        res = robot()
        res.set(x, y, orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        return res
    
    def Gaussian(self, mu, sigma, x):
        
        # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
        return exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(2.0 * pi * (sigma ** 2))
    
    
    def measurement_prob(self, measurement):
        
        # calculates how likely a measurement should be
        
        prob = 1.0;
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            prob *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return prob
    
    
    
    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))



def eval(r, p):
    sum = 0.0;
    for i in range(len(p)): # calculate mean error
        dx = (p[i].x - r.x + (world_size/2.0)) % world_size - (world_size/2.0)
        dy = (p[i].y - r.y + (world_size/2.0)) % world_size - (world_size/2.0)
        err = sqrt(dx * dx + dy * dy)
        sum += err
    return sum / float(len(p))

In [ ]:
# Make a robot called myrobot that starts at
# coordinates 30, 50 heading north (pi/2).
my_robot = robot()
my_robot.set(30, 50, pi/2)

# Now add noise to your robot as follows:
# forward_noise = 5.0, turn_noise = 0.1,
# sense_noise = 5.0.
my_robot.set_noise(5.0, 0.1, 5.0)

# Have your robot turn clockwise by pi/2, move
# 15 m, and sense (also print sensed measurements)
my_robot = my_robot.move(-pi/2, 15)
print(my_robot.sense())


#Then have it turn clockwise
# by pi/2 again, move 10 m, and sense again (also print sensed measurements).
my_robot = my_robot.move(-pi/2, 10)
print(my_robot.sense())

In [5]:
from random import randrange

#pick a particle index based on beta passed and weights
def pick_particle_index(index, w_p, beta):
    while w_p[index] < beta:
        beta -= w_p[index]
        index = (index + 1) % len(w_p);
        
    return index, beta

def resample_particles(p, w):
    #Generate a random index
    index = int(random.random() * len(p));
    #find maximum particle weight
    w_max = max(w)

    beta = 0
    p3 = []

    for i in range(len(p)):
        #set beta to any number between 
        beta += random.random() * 2 * w_max
        index, beta = pick_particle_index(index, w, beta)
        p3.append(p[index])
        
    return p3

# Each particle should turn by 
# and then move by (as did the robot myrobot above)
def move_particles(p, heading, forward):
    p2 = []
    for i in range(len(p)):
        p2.append(p[i].move(heading, forward))

    return p2

#calculate mismatch (or likelihood of a measurement) between predicted measurement of each particle p[i]
#and actual measurement Z that the robot (myrobot) sensed
#the lesser the mismatch, the great the likelihood probability of the measurement
def calculate_weights(p, Z):
    w = []
    for i in range(len(p)):
        w.append(p[i].measurement_prob(Z))
        
    return w

In [17]:
#create a robot and move it
myrobot = robot()
myrobot = myrobot.move(0.1, 5.0)
Z = myrobot.sense()

#create a list of 1000 particles (robot class objects, randomly initialized automatically)
#each particle is a vector = [x-coord, y-coord, heading-direction angle]
N = 1000
p = []
for i in range(N):
    x = robot()
    x.set_noise(0.05, 0.05, 5.0)
    p.append(x)

T = 10
for i in range(T):
    #move the robot and sense measurement
    myrobot = myrobot.move(0.1, 5.0)
    Z = myrobot.sense()
    
    #move the particles accordingly and 
    #calculate their weights. 
    p = move_particles(p, 0.1, 5.0)
    w = calculate_weights(p, Z)
    #resample particles according to their weights
    p = resample_particles(p, w)
    
    #calculate error
    error = eval(myrobot, p)
    print(error)

4.9176049839191265
7.644928875884506
7.060778408340283
6.8104767835526285
6.942908499887393
6.752593189983734
6.381536235007098
6.582107121822332
6.566275707807791
6.546549791629817


In [9]:
print(p)

[[x=83.208 y=30.282 orient=1.1815], [x=83.821 y=30.090 orient=1.1234], [x=82.271 y=30.526 orient=1.3215], [x=82.842 y=30.346 orient=1.2496], [x=83.356 y=30.133 orient=1.1846], [x=82.883 y=30.205 orient=1.2794], [x=82.735 y=30.312 orient=1.3080], [x=82.787 y=30.227 orient=1.2578], [x=82.841 y=30.357 orient=1.2497], [x=83.793 y=29.923 orient=1.1518], [x=82.418 y=30.442 orient=1.2804], [x=82.418 y=30.442 orient=1.2804], [x=82.178 y=30.627 orient=1.3358], [x=83.402 y=30.158 orient=1.1959], [x=83.402 y=30.158 orient=1.1959], [x=83.402 y=30.158 orient=1.1959], [x=83.746 y=29.949 orient=1.1495], [x=83.045 y=30.292 orient=1.2729], [x=83.381 y=30.065 orient=1.1855], [x=83.381 y=30.065 orient=1.1855], [x=82.340 y=30.487 orient=1.3292], [x=82.340 y=30.487 orient=1.3292], [x=82.971 y=30.254 orient=1.2431], [x=82.971 y=30.254 orient=1.2431], [x=83.591 y=30.076 orient=1.1968], [x=83.591 y=30.076 orient=1.1968], [x=83.292 y=30.200 orient=1.1948], [x=83.292 y=30.200 orient=1.1948], [x=83.505 y=30.055 

1.8470375965527845